# Review the Annotation file before publishing
---
__Notes__

This Notebook was created to review the manually curated annotations file that may have typos or other small errors.

## Load the Annotations
Drop the unused columns

In [ ]:
import pandas as pd 
data = pd.read_csv("./input_data/2.0_annotation.csv") 
#data.drop(data.columns[[6,7,8,9,10]], axis=1, inplace=True) 
#del data['Unnamed: 6']
data['index_col'] = data.index
data.head()

## Check for Empty Categories

In [ ]:
print("=========================================")
print("=========      Category 1      ==========")
print(data[data['Category 1'].isnull()]['Wormbase ID'])
print("=========================================")
print("=========      Category 2      ==========")
print(data[data['Category 2'].isnull()]['Wormbase ID'])
print("=========================================")
print("=========      Category 3      ==========")
print(data[data['Category 3'].isnull()]['Wormbase ID'])



## Check for Duplicate Wormbase ID's

In [ ]:
ids = data['Wormbase ID']
dup_ids = ids[ids.duplicated()]
#print(dup_ids)
df_dup_ids = pd.DataFrame({'Index':dup_ids.index, 'Wormbase ID':dup_ids.values})
print("Full List={} Dups={}".format(len(ids.index),len(dup_ids.index)))
df_dup_ids = df_dup_ids.sort_values('Wormbase ID')
total_dups = pd.DataFrame()
for i, row in df_dup_ids.iterrows():
    dup_row = data.loc[data['Wormbase ID'] == row['Wormbase ID']]
    dup_row = dup_row[['Sequence ID','Wormbase ID','Category 1']]
    total_dups = total_dups.append(dup_row)

pd.set_option('display.max_rows', 400)    
print(total_dups)
print('==================')
    

## Find Differences in Category 1 and Catergory 2
Each Category 1 Item should start with the Same text in Category 2
Check each category and sum the count if they match we are good
if they do not further investivation is needed.

In [ ]:
cat1 = data['Category 1'].unique()

series = data.groupby('Category 1')['index_col'].nunique()
unique_cat1 = pd.DataFrame({'Category 1':series.index, 'Count':series.values})
#print("type {}".format(unique_cat1))

print(" Cat1  Cat2  Diff Category 1")
for i, row in unique_cat1.iterrows():
    regex = "^{}".format(row['Category 1'])
    count = data.set_index('Category 2').filter(regex=regex, axis=0)['Wormbase ID'].count()
    if(count != row['Count']):
        print("{0: >5} {1: >5}  {2: >4} '{3}'".format(row['Count'],count,(count-row['Count']),row['Category 1']))

        

## Let's look at Extracellular material

In [ ]:
unique_2 = data.set_index('Category 2').filter(regex='^Extracellular material', axis=0)
#unique_2.set_index('index_col', inplace=True)
unique_1 = data.set_index('Category 1').filter(regex='^Extracellular material', axis=0)
print("Unique Cat 1={0:} Cat 2={1:}".format(len(unique_1.index),len(unique_2.index)))
count=0
for i, row in unique_1.iterrows():
    dup_row = unique_2.loc[unique_2['index_col'] == row['index_col']]
    if(dup_row.empty):
        count +=1
        print("{0:>2} {1:}".format(count,row[1]))
    

        

## Let's look at Transcription: other

In [ ]:
unique_2 = data.set_index('Category 2').filter(regex='^Transcription: other', axis=0)
#unique_2.set_index('index_col', inplace=True)
unique_1 = data.set_index('Category 1').filter(regex='^Transcription: other', axis=0)
print("{0: >5} {1: >5}".format(len(unique_1.index),len(unique_2.index)))
for i, row in unique_2.iterrows():
    dup_row = unique_1.loc[unique_1['index_col'] == row['index_col']]
    if(dup_row.empty):
        print(row)
   

## Find Differences in Category 2 and Catergory 3



In [ ]:
cat2 = data['Category 2'].unique()
#print(cat2)
series = data.groupby('Category 2')['Sequence ID'].nunique()
df = pd.DataFrame({'Category 2':series.index, 'Count':series.values})

print(" Cat2  Cat3  Diff Category 2")
for i, row in df.iterrows():
    regex = "^{}".format(row['Category 2'])
    count = data.set_index('Category 3').filter(regex=regex, axis=0)['Sequence ID'].count()
    if(count-row['Count']<0):
        print("{0: >5} {1: >5}  {2: >4} '{3}'".format(row['Count'],count,(count-row['Count']),row['Category 2']))
    if(count-row['Count']>0):
        print("{0: >5} {1: >5}  {2: >4} '{3}'".format(row['Count'],count,(count-row['Count']),row['Category 2']))

        

In [ ]:
unique_3 = data.set_index('Category 3').filter(regex='^Unknown', axis=0)
unique_2 = data.set_index('Category 2').filter(regex='^Unknown$', axis=0)
print("{0: >5} {1: >5}".format(len(unique_2.index),len(unique_3.index)))
for i, row in unique_2.iterrows():
        row_data = data.loc[data['index_col'] == row['index_col']]
        #print(row_data['Category 3'])
   

## List of unique categories in 1, 2, 3

In [ ]:
cat1 = data['Category 1'].unique()
for c in cat1:
    print(c)

In [ ]:
pd.set_option('display.max_rows', 400) 
cat2 = data['Category 2'].unique()
for c in cat2:
    print(c)

In [ ]:
cat3 = data['Category 3'].unique()
for c in cat3:
    print(c)